In [ ]:
# Image preprocessing

import cv2
import tifffile
import numpy as np
import matplotlib.pyplot as mpl
import os

def adjustClearMapImages(Basedir,adjustvalue):
    Basedir = Basedir
    dirs = "./ClearMap/data/{}/".format(Basedir)
    signal = os.path.join(dirs,"signal")
    autofluo = os.path.join(dirs,"autofluo")
    print(dirs)

    signallist = []
    autofluolist = []

    imglist = os.listdir(signal)
    imglist = sorted(imglist)
    autofluoimglist = os.listdir(autofluo)
    autofluoimglist = sorted(autofluoimglist)

    for imgname in imglist:
        img = cv2.imread(os.path.join(signal,imgname), cv2.IMREAD_ANYDEPTH)
        signallist += [img]
    for imgname in autofluoimglist:
        img = cv2.imread(os.path.join(autofluo,imgname), cv2.IMREAD_ANYDEPTH)
        autofluolist += [img]

    signallist = np.array(signallist)
    autofluolist = np.array(autofluolist)

    # flattening
    signallist_ravel = np.ravel(signallist)

    # top5%、bottom5%
    signallist_95 = np.percentile(signallist,95)
    signallist_5 = np.percentile(signallist,5)

    # Correction
    # Adjust so that the top 5% of values become 32767.

    adjustvalue = adjustvalue
    adjust = adjustvalue / signallist_95
    signallist_adjust = signallist * adjust
    signallist_adjust[signallist_adjust > 65535] = 65535
    signallist_adjust = signallist_adjust.astype("uint16")

    # compression
    signallist_adjust_resize = []
    autofluolist_resize = []

    for img in signallist_adjust:
        signallist_adjust_resize += [cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)]
    for img in autofluolist:
        autofluolist_resize += [cv2.resize(img, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_CUBIC)]
    signallist_adjust_resize = np.array(signallist_adjust_resize)
    autofluolist_resize = np.array(autofluolist_resize)

    signal_adjust = "ClearMap/data/{}/signal_adjust".format(Basedir)
    autofluo_adjust = "ClearMap/data/{}/autofluo_resize".format(Basedir)

    if not os.path.exists(signal_adjust):
        os.mkdir(signal_adjust)
    if not os.path.exists(autofluo_adjust):
        os.mkdir(autofluo_adjust)
    print(signal_adjust)
    print(autofluo_adjust)

    for i in range(len(signallist_adjust_resize)):
        img = signallist_adjust_resize[i]
        cv2.imwrite("{}/{:04}.tif".format(signal_adjust,i),img)
    for i in range(len(autofluolist_resize)):
        img = autofluolist_resize[i]
        cv2.imwrite("{}/{:04}.tif".format(autofluo_adjust,i),img)
    
    return
    
project = " " #project name
Basedirs = ["{}/{}".format(project,i) for i in os.listdir("./ClearMap/data/{}".format(project)) if not "." in i]
Basedirs = sorted(Basedirs)

for Basedir in Basedirs:
    adjustClearMapImages(Basedir,511) # Contrast setting value

In [ ]:
# ClearMap Analysis

import cv2
import tifffile
import gc
gc.collect()
import numpy as np
import pandas as pd
import matplotlib.pyplot as mpl
import os
np.random.seed(0)

project = " " #project name
if not os.getcwd() == "/username/ClearMap/ClearMap":
    Basedirs = ["{}/{}".format(project,i) for i in os.listdir("./ClearMap/data/{}".format(project)) if not "." in i]
    Basedirs = sorted(Basedirs)
print(Basedirs)

# Threshold for cell detection.
thresholds = [(30,900)]

os.chdir("/username/ClearMap/ClearMap")
import ClearMap.Visualization.Plot as plt


for Basedir in Basedirs[0:1]:
    BaseDirectory = os.path.join('/username/ClearMap/ClearMap/data',Basedir)
    execfile('./ClearMap/Scripts/practice_parameter_file_ver3.py')
    print(BaseDirectory)
    print(cFosFile)
    print(AutofluoFile)
    resampleData(**CorrectionResamplingParameterCfos);
    resampleData(**CorrectionResamplingParameterAutoFluo);
    resampleData(**RegistrationResamplingParameter);
    resultDirectory = alignData(**CorrectionAlignmentParameter);
    resultDirectory = alignData(**RegistrationAlignmentParameter)
    detectCells(**ImageProcessingParameter);

    for thresh in thresholds:
        thresholdmin = thresh[0]
        thresholdmax = thresh[1]

        points, intensities = io.readPoints(ImageProcessingParameter["sink"]);
        points, intensities = thresholdPoints(points, intensities, 
                                    threshold = (thresholdmin, thresholdmax), row = (3,3));
        io.writePoints(FilteredCellsFile, (points, intensities));
        pointSource= FilteredCellsFile[0];
        data = plt.overlayPoints(cFosFile, pointSource, pointColor = None, **cFosFileRange);
        # Export the coordinates identified as cells onto the original image.
        if thresh == (2,900) or thresh == (30,900):
            io.writeData(os.path.join(BaseDirectory, 'cells_check_{}_{}.tif'.format(thresholdmin,thresholdmax)), data);
        points = io.readPoints(CorrectionResamplingPointsParameter["pointSource"]);
        points = resamplePoints(**CorrectionResamplingPointsParameter);
        points = transformPoints(points, 
                                 transformDirectory = CorrectionAlignmentParameter["resultDirectory"],
                                 indices = False, resultDirectory = None);
        CorrectionResamplingPointsInverseParameter["pointSource"] = points;
        points = resamplePointsInverse(**CorrectionResamplingPointsInverseParameter);
        RegistrationResamplingPointParameter["pointSource"] = points;
        points = resamplePoints(**RegistrationResamplingPointParameter);
        points = transformPoints(points, 
                                 transformDirectory = RegistrationAlignmentParameter["resultDirectory"],
                                 indices = False, resultDirectory = None);
        io.writePoints(os.path.join(BaseDirectory,"cells_transformed_to_Atlas_{}_{}.npy".format(thresholdmin,thresholdmax)), points);
        cells_transformed_to_Atlas = np.load(os.path.join(BaseDirectory,"cells_transformed_to_Atlas_{}_{}.npy".format(thresholdmin,thresholdmax)))
        points = io.readPoints(os.path.join(BaseDirectory,"cells_transformed_to_Atlas_{}_{}.npy".format(thresholdmin,thresholdmax)))
        intensities = io.readPoints(FilteredCellsFile[1])
        vox = voxelize(points, AtlasFile, **voxelizeParameter);
        io.writeData(os.path.join(BaseDirectory, 'cells_heatmap_{}_{}.tif'.format(thresholdmin,thresholdmax)), vox.astype('int32'));
        ids, counts = countPointsInRegions(points, labeledImage = AnnotationFile,
                                           intensities = None);
        table = numpy.zeros(ids.shape,dtype=[  ('id','int64'),('counts','f8'),('name', 'a256')])
        table["id"] = ids;
        table["counts"] = counts;
        table["name"] = labelToName(ids);
        io.writeTable(os.path.join(BaseDirectory, 'Annotated_counts_{}_{}.csv'.format(thresholdmin,thresholdmax)), table);
        print(table)

In [ ]:
# Voxel, Region Statistics

import gc
gc.collect()
import numpy
import pandas
import matplotlib.pyplot as mpl
import os
numpy.random.seed(0)
baseDirectory = "/username/ClearMap"
ABAlabeledImage = '/username/ClearMap/ClearMap/Atlas/annotation_25_less228.nrrd';

# Project folder
folder = " "

if not os.getcwd() == "/username/ClearMap/ClearMap":
    Basedirs = ["{}/{}".format(folder,i) for i in os.listdir("./ClearMap/data/{}".format(folder)) if not "." in i]
    Basedirs = sorted(Basedirs)
print(Basedirs)

# Batch execution.
os.chdir("/username/ClearMap/ClearMap")

import ClearMap.Analysis.Statistics as stat
import ClearMap.Analysis.Tools.MultipleComparisonCorrection as mc
import ClearMap.Analysis.Label as lbl
import ClearMap.IO.IO as io
from ClearMap.Analysis.Label import labelToName
from ClearMap.Alignment import Resampling

# Cell detection threshold
thresholds = [(30,900)]
# p value
pthresholds = [0.001]

# group_names 
#TFOO：31℃-Fed
#TFAS：31℃-Fast
#COLD：16℃-Fed
#TFOO：16℃-Fast
group_names = [("TORP","TFOO"),("COLD","TFOO"),("TFAS","TFOO")]

for group_name in group_names:
    group1_name = group_name[0]
    group2_name = group_name[1]
    for thresh in thresholds:
        thresholdmin = thresh[0]
        thresholdmax = thresh[1]

        # Voxel Statistics
        savedir = os.path.join(baseDirectory, 'pvalue_heatmap/{}/{}vs{}_{}_{}'.format(folder,group1_name,group2_name,thresholdmin,thresholdmax))
        if not os.path.exists((savedir)):
            os.mkdir(savedir)
            
        group1_names = [name1 for name1 in Basedirs if group1_name in name1]
        group1 = ['/username/ClearMap/ClearMap/data/{}/cells_heatmap_{}_{}.tif'.format(name1,thresholdmin,thresholdmax) for name1 in group1_names];
        group2_names = [name2 for name2 in Basedirs if group2_name in name2]
        group2 = ['/username/ClearMap/ClearMap/data/{}/cells_heatmap_{}_{}.tif'.format(name2,thresholdmin,thresholdmax) for name2 in group2_names];
        g1 = stat.readDataGroup(group1);
        g2 = stat.readDataGroup(group2);

        g1m = numpy.mean(g1,axis = 0);
        io.writeData(os.path.join(savedir, '{}_mean_{}_{}.mhd'.format(group1_name,thresholdmin,thresholdmax)),Resampling.sagittalToCoronalData(g1m)[:,::-1,:]);
        g1s = numpy.std(g1,axis = 0);
        io.writeData(os.path.join(savedir, '{}_std_{}_{}.mhd'.format(group1_name,thresholdmin,thresholdmax)),Resampling.sagittalToCoronalData(g1s)[:,::-1,:]);
        g2m = numpy.mean(g2,axis = 0);
        io.writeData(os.path.join(savedir, '{}_mean_{}_{}.mhd'.format(group2_name,thresholdmin,thresholdmax)),Resampling.sagittalToCoronalData(g2m)[:,::-1,:]);
        g2s = numpy.std(g2,axis = 0);
        io.writeData(os.path.join(savedir, '{}_std_{}_{}.mhd'.format(group2_name,thresholdmin,thresholdmax)),Resampling.sagittalToCoronalData(g2s)[:,::-1,:]);

        for pthresh in pthresholds:
            pvals, psign = stat.tTestVoxelization(g1.astype('float'), g2.astype('float'),signed = True, pcutoff = pthresh);
            pvalscolor = stat.colorPValues(pvals, psign, positive = [0,1], negative = [1,0]);
            io.writeData(os.path.join(savedir, 'pvalues_{}vs{}_{}_{}_{}.tif'.format(group1_name,group2_name,thresholdmin,thresholdmax,pthresh)), Resampling.sagittalToCoronalData(pvalscolor.astype('float32'))[:,::-1,:]);

        # Region statistics
        savedir = os.path.join(baseDirectory, 'pvalue_list/{}/{}vs{}_{}_{}'.format(folder,group1_name,group2_name,thresholdmin,thresholdmax))
        if not os.path.exists((savedir)):
            os.mkdir(savedir)

        group1 = ['/username/ClearMap/ClearMap/data/{}/cells_transformed_to_Atlas_{}_{}.npy'.format(name1,thresholdmin,thresholdmax) for name1 in group1_names];
        group2 = ['/username/ClearMap/ClearMap/data/{}/cells_transformed_to_Atlas_{}_{}.npy'.format(name2,thresholdmin,thresholdmax) for name2 in group2_names];

        ids, counts1 = stat.countPointsGroupInRegions(group1, intensityGroup = None, returnIds = True, labeledImage = ABAlabeledImage, returnCounts = True, collapse = None); 
        counts2 = stat.countPointsGroupInRegions(group2, intensityGroup = None, returnIds = False, labeledImage = ABAlabeledImage, returnCounts = True, collapse = None);
        pvals, psign = stat.tTestPointsInRegions(counts1, counts2, pcutoff = None,signed = True, equal_var = True);
        qvals = mc.estimateQValues(pvals);
        table = numpy.zeros(ids.shape,dtype=[('id','int64'),('p-value','f8'),('name', 'a256')])
        table["id"] = ids;
        table["p-value"] = pvals;
        table["name"] = labelToName(ids);
        io.writeTable(os.path.join(savedir, 'regions_pvals_{}vs{}_{}_{}_varequal.csv'.format(group1_name,group2_name,thresholdmin,thresholdmax)), table);

        ids, counts1 = stat.countPointsGroupInRegions(group1, intensityGroup = None, returnIds = True, labeledImage = ABAlabeledImage, returnCounts = True, collapse = None); 
        counts2 = stat.countPointsGroupInRegions(group2, intensityGroup = None, returnIds = False, labeledImage = ABAlabeledImage, returnCounts = True, collapse = None);
        pvals, psign = stat.tTestPointsInRegions(counts1, counts2, pcutoff = None,signed = True, equal_var = False);
        qvals = mc.estimateQValues(pvals);
        table = numpy.zeros(ids.shape,dtype=[('id','int64'),('p-value','f8'),('name', 'a256')])
        table["id"] = ids;
        table["p-value"] = pvals;
        table["name"] = labelToName(ids);
        io.writeTable(os.path.join(savedir, 'regions_pvals_{}vs{}_{}_{}_varnotequal.csv'.format(group1_name,group2_name,thresholdmin,thresholdmax)), table);